### LlamaIndexの前準備

In [1]:
# パッケージのインストール
!pip install llama-index==0.10.39
!pip install llama-index-llms-gemini
!pip install llama-index-embeddings-huggingface

  Using cached llama_index_core-0.10.68.post1-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.10.68.post1-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.22
    Uninstalling llama-index-core-0.11.22:
      Successfully uninstalled llama-index-core-0.11.22
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-postprocessor-flag-embedding-reranker 0.2.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-llms-gemini 0.3.7 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
  Using cached llam

In [4]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from tqdm.notebook import tqdm

load_dotenv()
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [6]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# LLMの準備
Settings.llm = Gemini(
    model_name="models/gemini-1.5-flash",
    safety_settings={
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT" : "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT" : "BLOCK_NONE"
    }
)

# 埋め込みモデルの準備
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3"
)

INFO:datasets:PyTorch version 2.5.1 available.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/11" 200 0
DEBUG:filelock:Attempting to acquire lock 281471091186512 on /tmp/llama_index/.locks/models--BAAI--bge-m3/952a9b81c0bfd99800fabf352f69c7ccd46c5e43.lock
DEBUG:filelock:Lock 281471091186512 acquired on /tmp/llama_index/.locks/models--BAAI--bge-m3/952a9b81c0bfd99800fabf352f69c7ccd46c5e43.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /BAAI/bge-m3/resolve/main/modules.json HTTP/11" 200 349
DEBUG:filelock:Attempting to release lock 281471091186512 on /tmp/llama_index/.locks/models--BAAI--bge-m3/952a9b81c0bfd99800fabf352f69c7ccd46c5e43.lock
DEBUG:filelock:Lock 281471091186512 released on /tmp/llama_index/.locks/models--BAAI--bge-m3/952a9

### LLMのカスタマイズ

### LLMをGemini 1.0 Proに切り替え

In [8]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini

# LLMの設定
Settings.llm = Gemini(
    model_name="models/gemini-1.0-pro",
    temperature=0.3,
    safety_settings={
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT" : "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT" : "BLOCK_NONE"
    }
)

In [9]:
# LLMの動作確認
response = Settings.llm.complete("日本一高い山は？")
print(response)

富士山


### LLMをGPT-4に切り替え

In [ ]:
# パッケージのインストール
!pip install llama-index-llms-openai

In [ ]:
import os
from google.colab import userdata

# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

# LLMの設定
Settings.llm = OpenAI(
    model="gpt-4-turbo",
    temperature=0.3
)

In [ ]:
# LLMの動作確認
response = Settings.llm.complete("日本一高い山は？")
print(response)

### 埋め込みモデルのカスタマイズ

### 埋め込みモデルをtext-embedding-3-smallに切り替え

In [ ]:
# パッケージのインストール
!pip install llama-index-embeddings-openai

In [ ]:
import os
from google.colab import userdata

# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

# 埋め込みモデルの設定
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
)

In [ ]:
# 埋め込みモデルの動作確認
embeddings = Settings.embed_model.get_text_embedding("日本一高い山は？")
print(len(embeddings))
print(embeddings[:5])

### トークナイザーのカスタマイズ

In [1]:
from llama_index.core import Settings
import tiktoken

# トークナイザーの設定
Settings.tokenizer = tiktoken.encoding_for_model("gpt-4-turbo").encode

In [ ]:
# トークナイザーの動作確認
tokens = Settings.tokenizer("こんにちは、世界！")
print(tokens)

### テキストスプリッターのカスタマイズ

In [28]:
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

# テキストスプリッターの設定
Settings.text_splitter = SentenceSplitter(
    chunk_size=20,  # テスト用に極端に小さくしています
    chunk_overlap=0,
    paragraph_separator="\n\n",  # 段落の区切り文字に"\n\n"を指定
)

In [29]:
# 分割するテキストの準備
text = '''1章のコンテキスト

2章のコンテキスト

3章のコンテキスト'''

# テキストスプリッターの動作確認
Settings.text_splitter.split_text(text)

['1章のコンテキスト\n\n2章のコンテキスト', '3章のコンテキスト']

In [35]:
# テキストスプリッターをデフォルトに戻す
Settings.text_splitter = SentenceSplitter()

### 取得するノード数の調整

In [6]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# LLMの準備
Settings.llm = Gemini(
    model_name="models/gemini-1.5-flash",
    safety_settings={
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT" : "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT" : "BLOCK_NONE"
    }
)

# 埋め込みモデルの準備
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3"
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config_sentence_transformers.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/README.md HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/sentence_bert_config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/model.safetensors HTTP/11" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connectio

In [9]:
from llama_index.core import SimpleDirectoryReader

# ドキュメントの読み込み (dataフォルダにドキュメントを配置しておきます)
documents = SimpleDirectoryReader("data").load_data()

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 7
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin1.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin2.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin3.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin4.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin5.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin6.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin7.txt


In [10]:
from llama_index.core import VectorStoreIndex

# インデックスの作成
index = VectorStoreIndex.from_documents(documents)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めな...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するた...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第4章：裏切り
ある日、オオカミの裏切りが発覚する。彼は帝国に情報を売っていたのだ。
アズキは...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第5章：決戦
ついに、アズキとオオカミは帝国の本拠地へ潜入する。そこで、彼らは帝国の恐るべき計...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第6章：希望の光
激しい戦いの末、アズキとオオカミはシステムを破壊することに成功する。帝国は崩...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


In [11]:
# クエリエンジンの作成
query_engine = index.as_query_engine(
    similarity_top_k=4,
)

In [12]:
# 質問応答
print(query_engine.query("アズキの年齢は？"))

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.13it/s]

DEBUG:llama_index.core.indices.utils:> Top 4 nodes:
> [Node e477f34b-94e7-4b45-9278-f05193540b5f] [Similarity score:             0.544501] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
> [Node 1b130908-8383-4dd3-a97a-6d1847943b5e] [Similarity score:             0.510709] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...
> [Node 5cbe7dee-07ce-44af-a673-fd7c353d3376] [Similarity score:             0.471858] 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑圧された人々を助け、自由と正義のために戦い続ける。
アズキは、ネオン街の影から生まれ、希望の光となっ...
> [Node a4ba942c-e46e-4d9d-a7b1-423ccdb80620] [Similarity score:             0.458938] 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するために、様々なハッキング作戦を実行していく。
しかし、帝国も黙っていない。アズキたちを追跡し、次々と罠...


アズキは16歳です。 



### Rerankerの設定

In [2]:
# Rerankerパッケージのインストール
!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install FlagEmbedding

  Using cached FlagEmbedding-1.3.2-py3-none-any.whl
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached ir_datasets-0.5.9-py3-none-any.whl.metadata (12 kB)
  Using cached pyarrow-18.0.0-cp311-cp311-manylinux_2_28_aarch64.whl.metadata (3.3 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.17-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (6.7 kB)
  Using cached inscriptis-2.5.0-py3-none-any.wh

In [7]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

# Rerankerの準備
rerank = FlagEmbeddingReranker(
    model="BAAI/bge-reranker-v2-m3",
    use_fp16=True,
    top_n=2
)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-reranker-v2-m3/resolve/main/tokenizer_config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-reranker-v2-m3/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-reranker-v2-m3/resolve/main/model.safetensors HTTP/11" 302 0
DEBUG:filelock:Attempting to acquire lock 281471022791120 on /root/.cache/huggingface/hub/.locks/models--BAAI--bge-reranker-v2-m3/d9e3e081faff1eefb84019509b2f5558fd74c1a05a2c7db22f74174fcedb5286.lock
DEBUG:filelock:Lock 281471022791120 acquired on /root/.cache/huggingface/hub/.locks/models--BAAI--bge-reranker-v2-m3/d9e3e081faff1eefb84019509b2f5558fd74c1a05a2c7db22f74174fcedb5286.lock
DEBUG:urllib3.connectionpool:https://cdn-lfs-us-1.hf.co:443 "GET /repos/fb/3f/fb3fe87288ff7667a58d0dac0de359a1ea242a8cb977a35823603c68613285dc/d9e3e081faff1eefb84019509b2f5558fd74c1a05a2c7db22f74174fcedb5286?response

In [16]:
# クエリエンジンの準備
query_engine = index.as_query_engine(
    similarity_top_k=4,
    node_postprocessors=[rerank]
)

In [17]:
# 質問応答
response = query_engine.query("アズキの年齢は？")
print(response)

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.08it/s]

DEBUG:llama_index.core.indices.utils:> Top 4 nodes:
> [Node e477f34b-94e7-4b45-9278-f05193540b5f] [Similarity score:             0.544501] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
> [Node 1b130908-8383-4dd3-a97a-6d1847943b5e] [Similarity score:             0.510709] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...
> [Node 5cbe7dee-07ce-44af-a673-fd7c353d3376] [Similarity score:             0.471858] 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑圧された人々を助け、自由と正義のために戦い続ける。
アズキは、ネオン街の影から生まれ、希望の光となっ...
> [Node a4ba942c-e46e-4d9d-a7b1-423ccdb80620] [Similarity score:             0.458938] 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するために、様々なハッキング作戦を実行していく。
しかし、帝国も黙っていない。アズキたちを追跡し、次々と罠...



pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 874.18it/s]


アズキは16歳です。 



In [15]:
# 取得したノードの確認
for node in response.source_nodes:
     print(node.get_text())
     print("--")

第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「帝国」の秘密情報。帝国は街を支配し、人々を搾取していた。アズキたちは、そんな帝国に立ち向かうレジスタンスの尖兵だった。
しかし、アズキは罠に嵌められた。背後から襲い掛かる帝国の警備隊。アズキは必死に逃げ回るが、追いつかれ、捕らわれてしまう。
--
第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑圧された人々を助け、自由と正義のために戦い続ける。
アズキは、ネオン街の影から生まれ、希望の光となって世界を照らす存在となった。
--
